In [ ]:
import os
#os.environ["OMP_NUM_THREADS"] = "1"
#os.environ["PYTENSOR_FLAGS"] = "compiledir=./pytensor_cache,mode=FAST_COMPILE,optimizer=None"

In [ ]:
import numpy as np
import pandas as pd
import yaml
np.random.seed(42)
import os
from resolve.polynomial_chaos_expansion import PCEMultiFidelityModel
import matplotlib.pyplot as plt


In [ ]:
# Set the path to the yaml settings file here
path_to_settings = "../"
with open(f"{path_to_settings}/settings.yaml", "r") as f:
    config_file = yaml.safe_load(f)

version       = config_file["path_settings"]["version"]
path_out_cnp  = config_file["path_settings"]["path_out_cnp"]
path_out_cnp  += f"/{version}"
path_out_pce  = config_file["path_settings"]["path_out_pce"]
path_out_pce  += f"/{version}"

file_in       = f'{path_out_cnp}/cnp_{version}_output.csv'


In [ ]:

if not os.path.exists(path_out_pce):
   os.makedirs(path_out_pce)

# Set parameter name/x_labels -> needs to be consistent with data input file
x_labels        = config_file["simulation_settings"]["theta_headers"]
y_label_lf     = 'y_cnp'
y_err_label_lf = 'y_cnp_err'
y_label_mf     = 'y_cnp'
y_err_label_mf = 'y_cnp_err'
y_label_hf     = 'y_raw'


# Set parameter boundaries
xmin    = config_file["simulation_settings"]["theta_min"]
xmax    = config_file["simulation_settings"]["theta_max"]
x_fixed = config_file["simulation_settings"]["theta_fixed"]
parameters={}
for i,x in enumerate(x_labels):
   parameters[x]=[xmin[i],xmax[i]]


In [ ]:
df_data=pd.read_csv(file_in)
data = df_data.drop_duplicates()

In [ ]:

LF_cnp_noise=np.mean(df_data.loc[(df_data['fidelity']==0.) & (df_data['iteration']==0)][y_err_label_lf].to_numpy())
MF_cnp_noise=np.mean(df_data.loc[(df_data['fidelity']==1.) & (df_data['iteration']==0)][y_err_label_mf].to_numpy())
LF_sim_noise=np.std(df_data.loc[(df_data['fidelity']==0.) & (df_data['iteration']==0)][y_label_hf].to_numpy())
HF_sim_noise=np.std(df_data.loc[(df_data['fidelity']==1.) & (df_data['iteration']==0)][y_label_hf].to_numpy())

x_train_hf = df_data.loc[(df_data['fidelity']==1.) & (df_data['iteration']==0)][x_labels].to_numpy()
y_train_hf = df_data.loc[(df_data['fidelity']==1.) & (df_data['iteration']==0)][y_label_hf].to_numpy()

x_train_mf = df_data.loc[(df_data['fidelity']==1.) & (df_data['iteration']==0)][x_labels].to_numpy()
y_train_mf = df_data.loc[(df_data['fidelity']==1.) & (df_data['iteration']==0)][ y_label_lf].to_numpy()

x_train_lf = df_data.loc[(df_data['fidelity']==0.) & (df_data['iteration']==0)][x_labels].to_numpy()
y_train_lf = df_data.loc[(df_data['fidelity']==0.) & (df_data['iteration']==0)][ y_label_lf].to_numpy()

data_dict = {
    "lf": [x_train_lf, y_train_lf], 
    "mf": [x_train_mf, y_train_mf], 
    "hf": [x_train_hf, y_train_hf]
}

In [ ]:
priors = config_file["pce_settings"]["priors"]
priors["lf"]["sigma_y"] = LF_cnp_noise
#priors["hf"]["sigma_y"] = sigma_y
print("Fidelities used: ",list(priors.keys()))
polynomial_order = config_file["pce_settings"]["polynomial_order"]
print("Setting polynomial order to: ", polynomial_order)
print("Reading in priors from setting file:\n", priors)

# Set trainings data
trainings_data = {}
for key in priors:#
    trainings_data[key]=data_dict[key]


In [ ]:
# Initialize the multi-fidelity model
multi_fidelity_model = None
multi_fidelity_model = PCEMultiFidelityModel(trainings_data, priors, parameters,degree=polynomial_order)

In [ ]:
multi_fidelity_model.build_model()
multi_fidelity_model.sanity_check_of_basis()

In [ ]:
trace = multi_fidelity_model.run_inference(method="nuts", n_samples=2000,tune=1000, chains=4, cores=4, target_accept=0.95)

In [ ]:

os.makedirs(path_out_pce, exist_ok=True)

In [ ]:
multi_fidelity_model.save_trace(f"{path_out_pce}/trace.nc")

In [ ]:
multi_fidelity_model.summarize_sampler()

In [ ]:
fig = multi_fidelity_model.pareto_k(f"{path_out_pce}/pareto_k.png")

In [ ]:
fig = multi_fidelity_model.plot_diagnostics(f"{path_out_pce}/bfmi.png")

In [ ]:
multi_fidelity_model.print_table(f"{path_out_pce}/table.csv")

In [ ]:
fig = multi_fidelity_model.plot_pair(f"{path_out_pce}/pair.png")

In [ ]:
fig = multi_fidelity_model.plot_trace(f"{path_out_pce}/trace.png")